In [81]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib as plt
from vnstock import *

from typing import Literal
from datetime import datetime,timedelta

Theo payback time. Để định giá một cổ phiếu thì cần các chỉ số tài chính sau có mức tăng trưởng trên 10%
- Lợi nhuận sau thuế
- EPS: earningPerShare
- PE < 10: priceToEarning
- BVPS: bookValuePerShare
- ROE: roe
- Cash hoạt động kinh doanh > 0
- Tổng Nợ < 3 Sum (Lợi nhuận 3 năm)

In [140]:
class PayBackTime:
    def __init__(self, symbol: str = 'GEX',
                 report_range: Literal['yearly', 'quarterly'] = 'yearly',
                 is_all: bool = True,
                 window_size:int = 10,):

        self.window_size = window_size
        self.symbol = symbol
        self.report_range = report_range
        self.is_all = is_all
        self.sticker_price, self.MOS_price = None,None

        # Initialize indicator_df, income_df, and balance_df using class variables
        self.indicator_df = financial_ratio(symbol=self.symbol, report_range=self.report_range, is_all=self.is_all).T.reset_index() 
        self.income_df = financial_flow(symbol=self.symbol, report_type='incomestatement', report_range=self.report_range).reset_index()
        self.balance_df = financial_flow(symbol=self.symbol, report_type='balancesheet', report_range=self.report_range).reset_index()
        self.interest_cols = ['roe', 'earningPerShare', 'bookValuePerShare', 'revenue', 'grossProfit', 'capital']

    def calculate_interest_rate(self, df, column_name: str, report:bool = False):
        # Consider the first 10 rows using iloc
        selected_rows = df.head(self.window_size)
        
        # Calculate interest using the formula: (current_value - initial_value) / initial_value * 100
        initial_value = selected_rows[column_name].iloc[-1]
        current_value = selected_rows[column_name].iloc[0]
        # interest = ((current_value - initial_value) / initial_value) * 100
        time_length = selected_rows[column_name].count()
        interest =  ((current_value / initial_value) ** (1 / time_length) - 1) * 100
        
        if report: 
            if interest > 10:
                status = 'Good'
            elif interest < 0:
                status = 'Really Bad'
            else:
                status = 'Bad'
            
            # Print the result
            print(f'The {column_name}: {interest:.2f}% through {time_length} {self.report_range}: {status}')
            
        return interest
    
    def estimate_mean_interest(self, col_list: list, report:bool = False):
        mean_interest = []
        for col in col_list:
            interest = self.calculate_interest_rate(self.get_dataframe_by_column(col), column_name=col, report = report)
            mean_interest.append(interest)
        mean_interest = np.mean(mean_interest)
        return mean_interest
    
    def get_dataframe_by_column(self, column_name: str):
        """
        Get the DataFrame that contains the specified column.
        """
        dataframes = [self.indicator_df, self.income_df, self.balance_df]
        for df in dataframes:
            if column_name in df.columns:
                return df
        return None
    
    def check_debt(self):
        debt_col = 'debt'
        profit_col = 'grossProfit'
        debt = self.balance_df[debt_col].iloc[0]
        profit = self.income_df[profit_col].iloc[:2].sum()
        
        print(f'Debt: {debt} - Profit: {profit}')
        
        if debt > profit:
            print('Debt is more than profit.')
        elif debt < profit:
            print('Profit is more than debt.')
        else:
            print('Debt and profit are equal.')

    def get_current_eps(self):
        indicator_df_temp = financial_ratio(symbol=self.symbol, report_range='quarterly', is_all=False).T.reset_index().head(1) 
        return indicator_df_temp['earningPerShare'].iloc[0]
    
    def get_future_pe(self):
        return self.indicator_df['priceToEarning'].mean()

    def get_current_price(self):
        current_date = datetime.now().strftime('%Y-%m-%d')
        start_date = (datetime.strptime(current_date, "%Y-%m-%d") - timedelta(days=3)).strftime('%Y-%m-%d')

        print(f'Current date: {current_date}') 
        df =  stock_historical_data(symbol=self.symbol, 
                            start_date=start_date, 
                            end_date=current_date, resolution='1D', type='stock', beautify=False)
        current_price = df.close.iloc[0]
        return current_price


    def calculate_price(self, current_eps=None, future_pe=None, future_growth_rate=None,
                        MARR: int = 15, NoY: int = 10, MOS: int = 50):
        if current_eps is None:
            current_eps = self.get_current_eps()
        if future_pe is None:
            future_pe = self.get_future_pe()
        if future_growth_rate is None:
            future_growth_rate = self.estimate_mean_interest(col_list=self.interest_cols, report=False)

        # Convert the percentage rates to decimals
        MARR /= 100
        MOS /= 100
        future_growth_rate /= 100

        # Calculate the future EPS using the compound annual growth rate formula
        future_eps = current_eps * (1 + future_growth_rate) ** NoY
        # Calculate the future price using the future PE and future EPS
        future_price = future_pe * future_eps
        # Calculate the present value using the discounted cash flow formula
        self.sticker_price = future_price / (1 + MARR) ** NoY
        self.MOS_price = self.sticker_price*MOS
        return self.sticker_price, self.MOS_price

    
    def get_report(self):
        # Print the result
        print(f"\nCurrent EPS: {self.get_current_eps()}")
        print(f"Future PE: {self.get_future_pe():.2f}")
        print(f"Future Growth Rate: {self.estimate_mean_interest(col_list=self.interest_cols, report=True):.2f}%")
        print("-------------------------")
        if self.sticker_price is None:
            self.sticker_price, self.MOS_price = self.calculate_price()
        print(f"{self.symbol} - Sticker price: {self.sticker_price/1000:.2f} - MOS price: {(self.MOS_price)/1000:.2f} (nghìn VND)")
        print(f'Current price: {self.get_current_price()}')


In [141]:
pbt_generator = PayBackTime(symbol='DCM', report_range='yearly', window_size=10)

In [142]:
pbt_generator.check_debt()

Debt: 3561 - Profit: 8484
Profit is more than debt.


In [143]:
# pbt_generator.calculate_price()

In [144]:
pbt_generator.get_report()


Current EPS: 3050
Future PE: 7.05
The roe: 11.35% through 9 yearly: Good
The earningPerShare: 20.29% through 9 yearly: Good
The bookValuePerShare: 7.48% through 9 yearly: Bad
The revenue: 9.78% through 10 yearly: Bad
The grossProfit: 16.89% through 10 yearly: Good
The capital: 5.17% through 10 yearly: Bad
Future Growth Rate: 11.83%
-------------------------
DCM - Sticker price: 16.25 - MOS price: 8.13 (nghìn VND)
Current date: 2023-11-05
Current price: 29.85
